In [ ]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [ ]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (id);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [ ]:
workdir = r'/home/tari/Загрузки/taiga/Fontanka/Fontanka'
filename = 'Fontanka.db'
file_path = os.path.join(workdir, filename)
metatablepathlist = sorted([os.path.join(workdir,file) for file in os.listdir(workdir) if 'metatable' in file])
years = [str(y) for y in range(2005,2018)]
tagged = os.path.join(workdir,'texts_tagged')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepathlist[0], sep='\t', encoding='utf8')
meta = meta.fillna('')
meta.head()

In [ ]:
meta.columns

In [ ]:
if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [ ]:
meta.iloc[7].to_dict()

In [ ]:
for i in tqdm(range(len(meta))):
    values = meta.iloc[i].to_dict()

    
    columns = ', '.join(values.keys())
    
    #print(list(values.values()))
    placeholders = ', '.join('?' * len(values))
    sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
    #print(sql)
    try:
        cur.execute(sql, list(values.values()))

        valuest = {'id': values['textid'], 'text': unify.open_text(os.path.join(os.path.join(plain,years[0]), str(values['textid'])+".txt"))}

        columns = ', '.join(valuest.keys())
        placeholders = ', '.join('?' * len(valuest))

        sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql2, list(valuest.values()))
        try:
            valuest2 = {'id': values['textid'], 'text': unify.open_text(open(os.path.join(os.path.join(tagged,years[0]), str(values['textid'])+".txt"),'r', encoding='utf8').read())}
            columns = ', '.join(valuest2.keys())
            placeholders = ', '.join('?' * len(valuest2))
            sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
            cur.execute(sql3, list(valuest2.values()))
        except:
            valuest2 = {'id': values['textid'], 'text': ""}
            columns = ', '.join(valuest2.keys())
            placeholders = ', '.join('?' * len(valuest2))
            sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
            cur.execute(sql3, list(valuest2.values()))
        con.commit()
    except:
        continue